In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PyQt5.QtCore import Qt
import joblib
from scipy.signal import butter, filtfilt
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTabWidget, QWidget, QVBoxLayout, QHBoxLayout, QErrorMessage,
    QLabel, QLineEdit, QComboBox, QPushButton, QFileDialog, QFrame, QSpacerItem, QSizePolicy, 
    QGraphicsView, QGraphicsScene, QGraphicsRectItem
)
from PyQt5.QtWidgets import QLabel, QLineEdit, QComboBox, QPushButton, QFileDialog, QFrame, QSpacerItem, QSizePolicy

from PyQt5.QtGui import QPixmap, QImage, QBrush, QColor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt
import pickle

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("電磁結構模擬平台")
        self.setGeometry(100, 100, 600, 600)  # 設定整個頁面的大小

        self.tabs = QTabWidget() #建立分頁控制
        self.setCentralWidget(self.tabs)

        # Add tabs ##增加分頁的數量
        self.add_tabMLIN()
        self.add_tabGCPW()
        self.add_tabvertical_C1()
        self.add_tabvertical_C2()
        self.add_tabvertical_A1()
        self.add_tabvertical_A2()
    def update_layers(self):
        # Reset all layers to gray
        for layer in self.layers_tab1:
            layer.setBrush(QBrush(QColor(200, 200, 200)))

        # Highlight selected layers in red
        selected_layer1 = int(self.combo1_tab1.currentText()) - 1
        selected_layer2 = int(self.combo2_tab1.currentText()) - 1
        self.layers_tab1[selected_layer1*2].setBrush(QBrush(QColor(255, 0, 0)))
        self.layers_tab1[selected_layer2*2].setBrush(QBrush(QColor(0, 0, 255)))
    def update_layers2(self):
        # Reset all layers to gray
        for layer in self.layers_tab2:
            layer.setBrush(QBrush(QColor(200, 200, 200)))
        self.layers_tab2[0].setBrush(QBrush(QColor(255, 0, 0)))  # 自動點亮第 1 層為紅色
        self.layers_tab2[2].setBrush(QBrush(QColor(0, 0, 255)))  # 自動點亮第 3 層為藍色
        # Highlight selected layers in red
        # selected_layer1 = int(self.combo1_tab2.currentText()) - 1
        # selected_layer2 = int(self.combo2_tab2.currentText()) - 1
        # self.layers_tab2[selected_layer1*2].setBrush(QBrush(QColor(255, 0, 0)))
        # self.layers_tab2[selected_layer2*2].setBrush(QBrush(QColor(0, 0, 255)))
#################################################################################
##########                          MLIN                        #################
#################################################################################
    def add_tabMLIN(self):
        tab = QWidget()
        layout = QHBoxLayout()

        # Left layout
        left_layout = QVBoxLayout()

        # Part 1: Two input fields
        part1_layout = QVBoxLayout()

        input1_layout = QHBoxLayout()
        self.input1_tab1 = QLineEdit()
        part1_layout.addWidget(QLabel("起始頻率:"))##記得這邊的add 式介面產生的順序
        input1_layout.addWidget(self.input1_tab1)
        input1_layout.addWidget(QLabel("GHz"))  # 添加 GHz 單位
        part1_layout.addLayout(input1_layout)

        input2_layout = QHBoxLayout()
        self.input2_tab1 = QLineEdit()
        part1_layout.addWidget(QLabel("截止頻率:"))
        input2_layout.addWidget(self.input2_tab1)
        input2_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input2_layout)
        part1_layout.addWidget(QLabel("中高頻範圍:37GHz~41GHz"))
        part1_layout.addWidget(QLabel("高頻範圍:57GHz~63GHz"))  
        part1_layout.addWidget(QLabel("最小單位為0.01GHz")) 
        left_layout.addLayout(part1_layout)

        # Part 2: 下拉式選單
        part2_layout = QVBoxLayout()
        self.combo1_tab1 = QComboBox()
        self.combo2_tab1 = QComboBox()
        
        for i in "1234567":
            self.combo1_tab1.addItem(i)

        for i in "2345678":
            self.combo2_tab1.addItem(i)
            
        part2_layout.addWidget(QLabel("Layer 1:"))
        part2_layout.addWidget(self.combo1_tab1)
        part2_layout.addWidget(QLabel("Layer 2:"))
        part2_layout.addWidget(QLabel("第二層數字必須大於第一層")) 
        part2_layout.addWidget(self.combo2_tab1)
        left_layout.addLayout(part2_layout)

        #part3_layout = QHBoxLayout()
        part3_layout = QHBoxLayout()

        input3_layout = QHBoxLayout()
        self.input3_tab1 = QLineEdit()
        part3_layout.addWidget(QLabel("W ( 線寬 ):"))
        part3_layout.addWidget(QLabel("線寬範圍:0.5~1.5mm"))
        input3_layout.addWidget(self.input3_tab1)
        input3_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input3_layout)
        left_layout.addLayout(part3_layout)

        #part6_layout : line length ()
        part6_layout = QHBoxLayout()

        input6_layout = QHBoxLayout()
        self.input6_tab1 = QLineEdit()
        self.input6_tab1.setText(str(10.0))
        part6_layout.addWidget(QLabel("L ( 線長 ):"))
        part6_layout.addWidget(QLabel("初始設定 10mm"))
        input6_layout.addWidget(self.input6_tab1)
        input6_layout.addWidget(QLabel("mm"))
        part6_layout.addLayout(input6_layout)
        left_layout.addLayout(part6_layout)
        
        # Part 4: Calculate button
        self.calc_button_tab1 = QPushButton("Start")
        left_layout.addWidget(self.calc_button_tab1)
        
        # Part 5: Add image display in the bottom left
        self.scene_tab1 = QGraphicsScene()  # Ensure it's unique for MLIN
        self.view_tab1 = QGraphicsView(self.scene_tab1)  # Ensure it's unique for MLIN
        self.layers_tab1 = []  # Ensure it's unique for MLIN
        rectangle_sizes = [
            (200, 5), (200, 15), (200, 5), (200, 15),
            (200, 5), (200, 25), (200, 5), (200, 15), (200, 5),(200, 25),
            (200, 5), (200, 15), (200, 5), (200, 15),
            (200, 5)
        ]

        current_y = 0
        for width, height in rectangle_sizes:
            rect = QGraphicsRectItem(0, current_y, width, height)
            rect.setBrush(QBrush(QColor(200, 200, 200)))
            self.scene_tab1.addItem(rect)
            self.layers_tab1.append(rect)
            current_y += height

        left_layout.addWidget(self.view_tab1)

        # Connect dropdown changes to layer highlighting
        self.combo1_tab1.currentIndexChanged.connect(self.update_layers)
        self.combo2_tab1.currentIndexChanged.connect(self.update_layers)

        # Add a spacer to push the button to the bottom
        left_layout.addItem(QSpacerItem(20, 40, QSizePolicy.Maximum))

        # Right layout
        right_layout = QVBoxLayout()
        self.image_labels_tab1 = [[QLabel() for _ in range(2)] for _ in range(2)]#建立右側的圖片位置
        for row in self.image_labels_tab1:
            row_layout = QHBoxLayout()
            for label in row:
                label.setFrameStyle(QFrame.Box | QFrame.Plain)
                label.setFixedSize(400, 250)  # 右邊的圖的size
                row_layout.addWidget(label)
            right_layout.addLayout(row_layout)

        # Combine left and right layouts
        layout.addLayout(left_layout, 4)
        layout.addLayout(right_layout, 6)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "MLIN")

        # Connect button signal to slot
        self.calc_button_tab1.clicked.connect(lambda: self.start_calculation("MLIN"))

#################################################################################
##########                          GCPW                      ###################
#################################################################################
    def add_tabGCPW(self): 
        tab = QWidget()
        layout = QHBoxLayout()

        # Left layout
        left_layout = QVBoxLayout()

        # Part 1: Two input fields
        part1_layout = QVBoxLayout()

        input1_layout = QHBoxLayout()
        self.input1_tab2 = QLineEdit()
        part1_layout.addWidget(QLabel("起始頻率:"))
        input1_layout.addWidget(self.input1_tab2)
        input1_layout.addWidget(QLabel("GHz"))  # 添加 GHz 單位
        part1_layout.addLayout(input1_layout)

        input2_layout = QHBoxLayout()
        self.input2_tab2 = QLineEdit()
        part1_layout.addWidget(QLabel("截止頻率:"))
        input2_layout.addWidget(self.input2_tab2)
        input2_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input2_layout)
        part1_layout.addWidget(QLabel("中高頻範圍:37GHz~41GHz"))
        part1_layout.addWidget(QLabel("高頻範圍:57GHz~63GHz"))  
        part1_layout.addWidget(QLabel("最小單位為0.01GHz")) 
        left_layout.addLayout(part1_layout)

        # Part 2: Two dropdown menus
        part2_layout = QVBoxLayout()
        self.combo1_tab2 = QComboBox()
        self.combo2_tab2 = QComboBox()
        for i in "1234567":
            self.combo1_tab2.addItem(i)

        for i in "2345678":

            self.combo2_tab2.addItem(i)
            
        part2_layout.addWidget(QLabel("Layer 1:"))
        part2_layout.addWidget(self.combo1_tab2)
        part2_layout.addWidget(QLabel("Layer 2:"))
        part2_layout.addWidget(self.combo2_tab2)
        left_layout.addLayout(part2_layout)

        #part3_layout = QHBoxLayout()
        part3_layout = QHBoxLayout()

        input3_layout = QHBoxLayout()
        self.input3_tab2 = QLineEdit()
        part3_layout.addWidget(QLabel("W ( 線寬 ):"))
        part3_layout.addWidget(QLabel("線寬範圍:0.5~1.0mm"))
        input3_layout.addWidget(self.input3_tab2)
        input3_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input3_layout)

        input4_layout = QHBoxLayout()
        self.input4_tab2 = QLineEdit()
        part3_layout.addWidget(QLabel("S ( 槽寬 ):"))
        part3_layout.addWidget(QLabel("線寬範圍:0.2~0.5mm"))
        input4_layout.addWidget(self.input4_tab2)
        input4_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input4_layout)
        left_layout.addLayout(part3_layout)

        #part6_layout : line length ()
        part6_layout = QHBoxLayout()

        input6_layout = QHBoxLayout()
        self.input6_tab2 = QLineEdit()
        self.input6_tab2.setText(str(10.0)) #設定初始值
        part6_layout.addWidget(QLabel("L ( 線長 ):"))
        part6_layout.addWidget(QLabel("初始設定 10mm"))
        input6_layout.addWidget(self.input6_tab2)
        input6_layout.addWidget(QLabel("mm"))
        part6_layout.addLayout(input6_layout)
        left_layout.addLayout(part6_layout)

        
        # Part 4: Calculate button
        self.calc_button_tab2 = QPushButton("Start")
        left_layout.addWidget(self.calc_button_tab2)

        
        # Part 5: Add image display in the bottom left
        self.scene_tab2 = QGraphicsScene()  # Ensure it's unique for GCPW
        self.view_tab2 = QGraphicsView(self.scene_tab2)  # Ensure it's unique for GCPW
        self.layers_tab2 = []  # Ensure it's unique for GCPW

        rectangle_sizes = [
            (200, 5), (200, 15), (200, 5), (200, 15),
            (200, 5), (200, 25), (200, 5), (200, 15), (200, 5), (200, 25),
            (200, 5), (200, 15), (200, 5), (200, 15),
            (200, 5)
        ]

        current_y = 0
        for width, height in rectangle_sizes:
            rect = QGraphicsRectItem(0, current_y, width, height)
            rect.setBrush(QBrush(QColor(200, 200, 200)))
            self.scene_tab2.addItem(rect)
            self.layers_tab2.append(rect)
            current_y += height

        left_layout.addWidget(self.view_tab2)

        # Connect dropdown changes to layer highlighting
        self.combo1_tab2.currentIndexChanged.connect(self.update_layers2)
        self.combo2_tab2.currentIndexChanged.connect(self.update_layers2)
        
        # Add a spacer to push the button to the bottom
        left_layout.addItem(QSpacerItem(20, 40, QSizePolicy.Minimum))

        # Right layout
        right_layout = QVBoxLayout()
        self.image_labels_tab2 = [[QLabel() for _ in range(2)] for _ in range(2)]
        for row in self.image_labels_tab2:
            row_layout = QHBoxLayout()
            for label in row:
                label.setFrameStyle(QFrame.Box | QFrame.Plain)
                label.setFixedSize(500, 250)  # Set default image size
                row_layout.addWidget(label)
            right_layout.addLayout(row_layout)

        # Combine left and right layouts
        layout.addLayout(left_layout, 4)
        layout.addLayout(right_layout, 6)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "GCPW")

        # Connect button signal to slot
        self.calc_button_tab2.clicked.connect(lambda: self.start_calculation("GCPW"))

#################################################################################
##########                          C1                          #################
#################################################################################
    def add_tabvertical_C1(self):
        tab = QWidget()
        layout = QHBoxLayout()

        # Left layout
        left_layout = QVBoxLayout()

        # Part 1: Two input fields
        part1_layout = QVBoxLayout()

        input1_layout = QHBoxLayout()
        input2_layout = QHBoxLayout()
        self.input1_tab3 = QLineEdit()
        self.input2_tab3 = QLineEdit()
        part1_layout.addWidget(QLabel("起始頻率:"))
        input1_layout.addWidget(self.input1_tab3)
        input1_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input1_layout)
        part1_layout.addWidget(QLabel("截止頻率:"))
        input2_layout.addWidget(self.input2_tab3)
        input2_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input2_layout)
        part1_layout.addWidget(QLabel("中高頻範圍:37GHz~41GHz")) 
        part1_layout.addWidget(QLabel("最小單位為0.01GHz")) 


        left_layout.addLayout(part1_layout)

        # Part 3: Three input fields
        part3_layout = QVBoxLayout()
        
        input3_layout = QHBoxLayout()
        input4_layout = QHBoxLayout()
        input5_layout = QHBoxLayout()
        self.input3_tab3 = QLineEdit()
        self.input4_tab3 = QLineEdit()
        self.input5_tab3 = QLineEdit()
        part3_layout.addWidget(QLabel("Via  (0.1~0.25mm)"))
        input3_layout.addWidget(self.input3_tab3)
        input3_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input3_layout)
        part3_layout.addWidget(QLabel("Pad  (0.6~1mm)"))
        input4_layout.addWidget(self.input4_tab3)
        input4_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input4_layout)
        part3_layout.addWidget(QLabel("Via shield (1.3~1.7mm)"))
        input5_layout.addWidget(self.input5_tab3)
        input5_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input5_layout)
        left_layout.addLayout(part3_layout)

        # Part 4: Calculate button
        self.calc_button_tab3 = QPushButton("Start")
        left_layout.addWidget(self.calc_button_tab3)
        
        # Part 5: Add image display in the bottom left (結構圖)
        self.left_image_label_tab1 = QLabel()
        self.left_image_label_tab1.setFrameStyle(QFrame.Box | QFrame.Plain)
        self.left_image_label_tab1.setFixedSize(500, 250)  # Set default image size
        left_layout.addWidget(self.left_image_label_tab1)

        # Load and display the image
        pixmap = QPixmap("validation/C1.jpg")
        self.left_image_label_tab1.setPixmap(pixmap.scaled(500, 250, aspectRatioMode=True))

        # Add a spacer to push the button to the bottom
        left_layout.addItem(QSpacerItem(20, 40, QSizePolicy.Minimum))

        # Right layout
        right_layout = QVBoxLayout()
        self.image_labels_tab3 = [QLabel() for _ in range(2)]
        for label in self.image_labels_tab3:
            label.setFrameStyle(QFrame.Box | QFrame.Plain)
            label.setFixedSize(500, 300)  # Set default image size
            right_layout.addWidget(label)

        # Combine left and right layouts
        layout.addLayout(left_layout, 4)
        layout.addLayout(right_layout, 6)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "C1")

        # Connect button signal to slot
        self.calc_button_tab3.clicked.connect(lambda: self.start_calculation("C1"))

#################################################################################
##########                          C2                          #################
#################################################################################
    def add_tabvertical_C2(self):
        tab = QWidget()
        layout = QHBoxLayout()

        # Left layout
        left_layout = QVBoxLayout()

        # Part 1: Two input fields
        part1_layout = QVBoxLayout()
        input1_layout = QHBoxLayout()
        input2_layout = QHBoxLayout()
        self.input1_tab4 = QLineEdit()
        self.input2_tab4 = QLineEdit()
        part1_layout.addWidget(QLabel("起始頻率:"))
        input1_layout.addWidget(self.input1_tab4)
        input1_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input1_layout)
        part1_layout.addWidget(QLabel("截止頻率:"))
        input2_layout.addWidget(self.input2_tab4)
        input2_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input2_layout)
        part1_layout.addWidget(QLabel("中高頻範圍:57GHz~63GHz")) 
        part1_layout.addWidget(QLabel("最小單位為0.01GHz")) 
        left_layout.addLayout(part1_layout)

        # Part 3: Three input fields
        part3_layout = QVBoxLayout()
        input3_layout = QHBoxLayout()
        input4_layout = QHBoxLayout()
        input5_layout = QHBoxLayout()
        self.input3_tab4 = QLineEdit()
        self.input4_tab4 = QLineEdit()
        self.input5_tab4 = QLineEdit()
        
        part3_layout.addWidget(QLabel("Via  (0.1~0.3mm)"))
        input3_layout.addWidget(self.input3_tab4)
        input3_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input3_layout)
        
        part3_layout.addWidget(QLabel("Pad  (0.2~0.6mm)"))
        input4_layout.addWidget(self.input4_tab4)
        input4_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input4_layout)
        
        part3_layout.addWidget(QLabel("Via shield (1.3~1.6mm)"))
        input5_layout.addWidget(self.input5_tab4)
        input5_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input5_layout)
        left_layout.addLayout(part3_layout)

        # Part 4: Calculate button
        self.calc_button_tab4 = QPushButton("Start")
        left_layout.addWidget(self.calc_button_tab4)
        
        # Part 5: Add image display in the bottom left
        self.left_image_label_tab1 = QLabel()
        self.left_image_label_tab1.setFrameStyle(QFrame.Box | QFrame.Plain)
        self.left_image_label_tab1.setFixedSize(500, 250)  # Set default image size
        left_layout.addWidget(self.left_image_label_tab1)

        # Load and display the image
        pixmap = QPixmap("validation/C2.jpg")
        self.left_image_label_tab1.setPixmap(pixmap.scaled(500, 250, aspectRatioMode=True))



        # Add a spacer to push the button to the bottom
        left_layout.addItem(QSpacerItem(20, 40, QSizePolicy.Minimum))

        # Right layout
        right_layout = QVBoxLayout()
        self.image_labels_tab4 = [QLabel() for _ in range(2)]
        for label in self.image_labels_tab4:
            label.setFrameStyle(QFrame.Box | QFrame.Plain)
            label.setFixedSize(500, 300)  # Set default image size
            right_layout.addWidget(label)

        # Combine left and right layouts
        layout.addLayout(left_layout, 4)
        layout.addLayout(right_layout, 6)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "C2")

        # Connect button signal to slot
        self.calc_button_tab4.clicked.connect(lambda: self.start_calculation("C2"))

    def add_tabvertical_A1(self):
        tab = QWidget()
        layout = QHBoxLayout()

        # Left layout
        left_layout = QVBoxLayout()

        # Part 1: Two input fields
        part1_layout = QVBoxLayout()

        input1_layout = QHBoxLayout()
        input2_layout = QHBoxLayout()
        self.input1_tab5 = QLineEdit()
        self.input2_tab5 = QLineEdit()
        part1_layout.addWidget(QLabel("起始頻率"))
        input1_layout.addWidget(self.input1_tab5)
        input1_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input1_layout)
        
        part1_layout.addWidget(QLabel("截止頻率:"))
        input2_layout.addWidget(self.input2_tab5)
        input2_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input2_layout)
        part1_layout.addWidget(QLabel("中高頻範圍:37GHz~41GHz")) 
        part1_layout.addWidget(QLabel("最小單位為0.01GHz")) 
        left_layout.addLayout(part1_layout)

        # Part 3: Three input fields
        part3_layout = QVBoxLayout()
        input3_layout = QHBoxLayout()
        input4_layout = QHBoxLayout()
        input5_layout = QHBoxLayout()
        self.input3_tab5 = QLineEdit()
        self.input4_tab5 = QLineEdit()
        self.input5_tab5 = QLineEdit()
        part3_layout.addWidget(QLabel("Via  (0.1~0.3mm)"))
        input3_layout.addWidget(self.input3_tab5)
        input3_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input3_layout)
        part3_layout.addWidget(QLabel("Pad  (0.6~0.8mm)"))
        input4_layout.addWidget(self.input4_tab5)
        input4_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input4_layout)
        part3_layout.addWidget(QLabel("Via shield (1.35~1.5mm)"))
        input5_layout.addWidget(self.input5_tab5)
        input5_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input5_layout)
        left_layout.addLayout(part3_layout)

        # Part 4: Calculate button
        self.calc_button_tab5 = QPushButton("Start")
        left_layout.addWidget(self.calc_button_tab5)
        
        # Part 5: Add image display in the bottom left
        self.left_image_label_tab1 = QLabel()
        self.left_image_label_tab1.setFrameStyle(QFrame.Box | QFrame.Plain)
        self.left_image_label_tab1.setFixedSize(500, 250)  # Set default image size
        left_layout.addWidget(self.left_image_label_tab1)

        # Load and display the image
        pixmap = QPixmap("validation/A1.jpg")
        self.left_image_label_tab1.setPixmap(pixmap.scaled(500, 250, aspectRatioMode=True))



        # Add a spacer to push the button to the bottom
        left_layout.addItem(QSpacerItem(20, 40, QSizePolicy.Minimum))

        # Right layout
        right_layout = QVBoxLayout()
        self.image_labels_tab5 = [QLabel() for _ in range(2)]
        for label in self.image_labels_tab5:
            label.setFrameStyle(QFrame.Box | QFrame.Plain)
            label.setFixedSize(500, 300)  # Set default image size
            right_layout.addWidget(label)

        # Combine left and right layouts
        layout.addLayout(left_layout, 4)
        layout.addLayout(right_layout, 6)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "A1")

        # Connect button signal to slot
        self.calc_button_tab5.clicked.connect(lambda: self.start_calculation("A1"))
    def add_tabvertical_A2(self):
        tab = QWidget()
        layout = QHBoxLayout()

        # Left layout
        left_layout = QVBoxLayout()

        # Part 1: Two input fields
        part1_layout = QVBoxLayout()
        input1_layout = QHBoxLayout()
        input2_layout = QHBoxLayout()
        self.input1_tab6 = QLineEdit()
        self.input2_tab6 = QLineEdit()
        part1_layout.addWidget(QLabel("起始頻率"))
        input1_layout.addWidget(self.input1_tab6)
        input1_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input1_layout)
        part1_layout.addWidget(QLabel("截止頻率:"))
        input2_layout.addWidget(self.input2_tab6)
        input2_layout.addWidget(QLabel("GHz"))
        part1_layout.addLayout(input2_layout)
        part1_layout.addWidget(QLabel("中高頻範圍:57GHz~63GHz")) 
        part1_layout.addWidget(QLabel("最小單位為0.01GHz")) 
        left_layout.addLayout(part1_layout)

        # Part 3: Three input fields
        part3_layout = QVBoxLayout()
        input3_layout = QHBoxLayout()
        input4_layout = QHBoxLayout()
        input5_layout = QHBoxLayout()
        self.input3_tab6 = QLineEdit()
        self.input4_tab6 = QLineEdit()
        self.input5_tab6 = QLineEdit()
        part3_layout.addWidget(QLabel("Via  (0.1~0.25mm)"))
        input3_layout.addWidget(self.input3_tab6)
        input3_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input3_layout)
        part3_layout.addWidget(QLabel("Pad  (0.6~0.8mm)"))
        input4_layout.addWidget(self.input4_tab6)
        input4_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input4_layout)
        part3_layout.addWidget(QLabel("Via shield (1.15~1.35mm)"))
        input5_layout.addWidget(self.input5_tab6)
        input5_layout.addWidget(QLabel("mm"))
        part3_layout.addLayout(input5_layout)
        left_layout.addLayout(part3_layout)

        # Part 4: Calculate button
        self.calc_button_tab6 = QPushButton("Start")
        left_layout.addWidget(self.calc_button_tab6)
        
        # Part 5: Add image display in the bottom left
        self.left_image_label_tab1 = QLabel()
        self.left_image_label_tab1.setFrameStyle(QFrame.Box | QFrame.Plain)
        self.left_image_label_tab1.setFixedSize(500, 250)  # Set default image size
        left_layout.addWidget(self.left_image_label_tab1)

        # Load and display the image
        pixmap = QPixmap("validation/A2.jpg")
        self.left_image_label_tab1.setPixmap(pixmap.scaled(500, 250, aspectRatioMode=True))



        # Add a spacer to push the button to the bottom
        left_layout.addItem(QSpacerItem(20, 40, QSizePolicy.Minimum))

        # Right layout
        right_layout = QVBoxLayout()
        self.image_labels_tab6 = [QLabel() for _ in range(2)]
        for label in self.image_labels_tab6:
            label.setFrameStyle(QFrame.Box | QFrame.Plain)
            label.setFixedSize(500, 300)  # Set default image size
            right_layout.addWidget(label)

        # Combine left and right layouts
        layout.addLayout(left_layout, 4)
        layout.addLayout(right_layout, 6)

        tab.setLayout(layout)
        self.tabs.addTab(tab, "A2")

        # Connect button signal to slot
        self.calc_button_tab6.clicked.connect(lambda: self.start_calculation("A2"))
###########################################################################################
#撈出數據
###########################################################################################
    def start_calculation(self, tab_name):
    # 蒐集輸入
        if tab_name == "MLIN": 
            if not self.input1_tab1.text() or not self.input2_tab1.text() or not self.input3_tab1.text()  : 
                self.set_black_images(self.image_labels_tab1)
                self.show_error_message("Please enter all parameters .")
            else :
                input1 = float(self.input1_tab1.text())
                input2 = float(self.input2_tab1.text())
                combo1 = self.combo1_tab1.currentText()
                combo2 = self.combo2_tab1.currentText()
                w_input = float(self.input3_tab1.text()) #轉換成float
                image_labels = self.image_labels_tab1
                L_input = float(self.input6_tab1.text())




                ###########################################################################################
                #中高頻的部分 ( 37~41 GHz )
                ###########################################################################################

                # 定義頻率範圍 -- 錯誤條件的限制

                if input1 >= 37 and input2 <= 41 and input2 > input1: 
                    freq_401 = np.arange(37,41.01,0.01)
                    freq_x_axis = np.arange(input1, input2 + 0.01, 0.01)
                    num_freq = freq_x_axis.shape[0] #需要多大的頻率範圍點數

                    # 層數 -- 錯誤條件的限制

                    if combo2 <= combo1:

                        self.set_black_images(self.image_labels_tab1)
                        self.show_error_message("Please enter a valid number for layer choose.")
                    else:


                        # W -- 錯誤條件的限制 

                        if w_input < 0.5 or w_input > 1.5 :
                            

                            self.show_error_message("Please enter a valid number for W.")
                            
                        else : 

                            w = w_input 

                            L = L_input * 0.001 #換算成公尺
                            
                            input_freq = np.arange(input1, input2+0.01, 0.01)
                            size_input = input_freq.shape[0]
                            
                            input_w=np.full(size_input,w)
                            input_data=np.array([input_w,input_freq]).T


                            # 定義要讀取的文件名稱
                            loaded_multi_output_model = joblib.load(f"Trained_model/model_MLIN_{combo1}_{combo2}_mid.joblib")
                            # loaded_multi_output_model = pickle.load(open(f'Trained_model/model_MLIN_{combo1}_{combo2}_mid.pkl', 'rb'))
                            predicted_output = loaded_multi_output_model.predict(input_data)

                            predicted_output[:,1] = 20 * np.log10(np.exp(-predicted_output[:,3] * L)) #計算s21和ALPHA的關係式 如果需要使用再啟用

                            print(predicted_output.shape)
                            fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                            cutoff_frequency = 10  # 設定截止頻率 (Hz)
                            order = 2  # 濾波器階數
                            b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                            # 對每個輸出結果進行低通濾波
                            filtered_output = np.zeros_like(predicted_output)
                            for i in range(predicted_output.shape[1]):
                                filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])
                            # 定義圖形標題 # 左上到右下
                            titles = ['S21(Insertion Loss)', 'β(Phase constant)', 'α(Attenuation constant)', 'Zo(Characteristic Impedance)'] 

                            # 迭代每個數據列並繪製相應的圖形
                            for j in range(4):  # 假設 data_frame 有4列數據需要繪製
                                fig, ax = plt.subplots()
                                ax.plot(input_freq, filtered_output[:, j+1], label=f'Simulation data')
                                ax.set_xlabel('Frequency (GHz)')
                                ax.set_ylabel('Magnitude')
                                ax.set_title(titles[j] ,fontsize=16)
                                ax.legend()

                                # 將圖形轉換為圖像數據
                                fig.canvas.draw()
                                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                plt.close(fig)

                                # 計算圖像的位置
                                row = j // 2
                                col = j % 2
                                height, width, channel = image.shape
                                bytesPerLine = 3 * width
                                q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                pixmap = QPixmap.fromImage(q_image)

                                # 設置圖像到對應的 QLabel 中
                                image_labels[row][col].setPixmap(pixmap.scaled(450, 300, Qt.KeepAspectRatio))
                            
                ###########################################################################################
                #高頻的部分 ( 57~63 GHz )
                ###########################################################################################

                # 定義頻率範圍 -- 錯誤條件的限制

                elif input1 >= 57 and input2 <= 63 and input2 > input1: 

                    # 層數 -- 錯誤條件的限制

                    if combo2 <= combo1:

                        self.set_black_images(self.image_labels_tab1)
                        self.show_error_message("Please enter a valid number for layer choose.")
                    else:


                        # W -- 錯誤條件的限制 

                        if w_input < 0.5 or w_input > 1.5 :
                            
                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for W.")
                            
                        else : 

                            w = w_input 

                            L = L_input * 0.001

                            input_freq = np.arange(input1, input2+0.01, 0.01)
                            size_input = input_freq.shape[0]
                            
                            input_w=np.full(size_input,w)
                            input_data=np.array([input_w,input_freq]).T


                            # 定義要讀取的文件名稱
                            loaded_multi_output_model = joblib.load(f"Trained_model/model_MLIN_{combo1}_{combo2}_high.joblib")
                            predicted_output = loaded_multi_output_model.predict(input_data)

                            predicted_output[:,1] = 20 * np.log10(np.exp(-predicted_output[:,3] * L))

                            fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                            cutoff_frequency = 10  # 設定截止頻率 (Hz)
                            order = 2  # 濾波器階數
                            b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                            # 對每個輸出結果進行低通濾波
                            filtered_output = np.zeros_like(predicted_output)
                            for i in range(predicted_output.shape[1]):
                                filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])
                            # 定義圖形標題 # 左上到右下
                            titles = ['S21(Insertion Loss)', 'β(Phase constant)', 'α(Attenuation constant)', 'Zo(Characteristic Impedance)']  

                            # 讀取數據文件



                            # 迭代每個數據列並繪製相應的圖形
                            for j in range(4):  # 假設 data_frame 有4列數據需要繪製
                                fig, ax = plt.subplots()
                                ax.plot(input_freq, filtered_output[:, j+1], label=f'Simulation data')
                                ax.set_xlabel('Frequency (GHz)')
                                ax.set_ylabel('Magnitude')
                                ax.set_title(titles[j], fontsize=16)
                                ax.legend()

                                # 將圖形轉換為圖像數據
                                fig.canvas.draw()
                                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                plt.close(fig)

                                # 計算圖像的位置
                                row = j // 2
                                col = j % 2
                                height, width, channel = image.shape
                                bytesPerLine = 3 * width
                                q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                pixmap = QPixmap.fromImage(q_image)

                                # 設置圖像到對應的 QLabel 中
                                image_labels[row][col].setPixmap(pixmap.scaled(450, 300, Qt.KeepAspectRatio))

                else:
                    self.set_black_images(self.image_labels_tab1)
                    self.show_error_message("Please enter a valid number for frequency.")
            


            

        elif tab_name == "GCPW":
            if not self.input1_tab2.text() or not self.input2_tab2.text() or not self.input3_tab2.text() or not self.input4_tab2.text()  : 
                self.set_black_images(self.image_labels_tab1)
                self.show_error_message("Please enter all parameters .")
            else :
                input1 = float(self.input1_tab2.text())
                input2 = float(self.input2_tab2.text())
                combo1 = self.combo1_tab2.currentText()
                combo2 = self.combo2_tab2.currentText()
                w_input = float(self.input3_tab2.text())
                s_input = float(self.input4_tab2.text())
                image_labels = self.image_labels_tab2
                L_input = float(self.input6_tab2.text())

                if input1 >= 37 and input2 <= 41 and input2 > input1: 

                    # 層數 -- 錯誤條件的限制

                    if combo2 <= combo1:

                        self.set_black_images(self.image_labels_tab2)
                        self.show_error_message("Please enter a valid number for layer choose.")
                    else:


                        if w_input < 0.5 or w_input > 1.0 :
                            
                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for W(線寬).")
                        else : 

                            w = w_input 

                            if  s_input < 0.2 or s_input > 0.5 :
                                
                                self.set_black_images(self.image_labels_tab1)
                                self.show_error_message("Please enter a valid number for S(槽寬).")
                            
                            else :

                                s = s_input

                                L = L_input * 0.001

                                image_labels = self.image_labels_tab2


                                input_freq = np.arange(input1, input2+0.01, 0.01)

                                size_input = input_freq.shape[0]
                                
                                input_s = np.full(size_input,s)

                                input_w = np.full(size_input,w)  
                                
                                input_data = np.array([input_s,input_w, input_freq]).T
                                
                                try : 
                                    loaded_multi_output_model = joblib.load(f"Trained_model/model_GCPW_{combo1}_{combo2}_mid.joblib")
                                except FileNotFoundError:
                                    self.set_black_images(self.image_labels_tab2)
                                    self.show_error_message("No corresponding model found for GCPW.")
                                    
                                
                                predicted_output = loaded_multi_output_model.predict(input_data)

                                predicted_output[:,1] = 20 * np.log10(np.exp(-predicted_output[:,3] * L))
                                
                                # 定義圖形標題 # 左上到右下
                                titles = ['S21(Insertion Loss)', 'β(Phase constant)', 'α(Attenuation constant)', 'Zo(Characteristic Impedance)'] 

                                fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                                cutoff_frequency = 10  # 設定截止頻率 (Hz)
                                order = 2  # 濾波器階數
                                b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                                # 對每個輸出結果進行低通濾波
                                filtered_output = np.zeros_like(predicted_output)
                                for i in range(predicted_output.shape[1]):
                                    filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])



                                # 迭代每個數據列並繪製相應的圖形
                                for j in range(4):  # 假設 data_frame 有4列數據需要繪製
                                    fig, ax = plt.subplots()
                                    ax.plot(input_freq, filtered_output[:, j+1], label=f'Simulation data')
                                    ax.set_xlabel('Frequency (GHz)')
                                    ax.set_ylabel('Magnitude')
                                    ax.set_title(titles[j], fontsize=16)
                                    ax.legend()

                                    # 將圖形轉換為圖像數據
                                    fig.canvas.draw()
                                    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                    plt.close(fig)

                                    # 計算圖像的位置
                                    row = j // 2
                                    col = j % 2
                                    height, width, channel = image.shape
                                    bytesPerLine = 3 * width
                                    q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                    pixmap = QPixmap.fromImage(q_image)

                                    # 設置圖像到對應的 QLabel 中
                                    image_labels[row][col].setPixmap(pixmap.scaled(450, 300, Qt.KeepAspectRatio))

                            
                            

                ###########################################################################################
                #高頻的部分 ( 57~63 GHz )
                ###########################################################################################

                # 定義頻率範圍 -- 錯誤條件的限制

                elif input1 >= 57 and input2 <= 63 and input2 > input1: 

                    # 層數 -- 錯誤條件的限制

                    if combo2 <= combo1:

                        self.set_black_images(self.image_labels_tab2)
                        self.show_error_message("Please enter a valid number for layer choose.")
                    else:


                        if w_input < 0.5 or w_input > 1.0 :
                            
                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for W(線寬).")
                        else : 

                            w = w_input 

                            if  s_input < 0.2 or s_input > 0.5 :
                                
                                self.set_black_images(self.image_labels_tab1)
                                self.show_error_message("Please enter a valid number for S(槽寬).")
                            
                            else :

                                s = s_input

                                L = L_input * 0.001

                                image_labels = self.image_labels_tab2


                                input_freq = np.arange(input1, input2+0.01, 0.01)

                                size_input = input_freq.shape[0]
                                
                                input_s = np.full(size_input,s)

                                input_w = np.full(size_input,w)  
                                
                                input_data = np.array([input_s,input_w, input_freq]).T

                                try : 
                                    loaded_multi_output_model = joblib.load(f"Trained_model/model_GCPW_{combo1}_{combo2}_high.joblib")
                                except FileNotFoundError:
                                    self.set_black_images(self.image_labels_tab2)
                                    self.show_error_message("No corresponding model found for GCPW")
                                
                                predicted_output = loaded_multi_output_model.predict(input_data)

                                predicted_output[:,1] = 20 * np.log10(np.exp(-predicted_output[:,3] * L))

                                # 定義圖形標題 # 左上到右下
                                titles = ['S21(Insertion Loss)', 'β(Phase constant)', 'α(Attenuation constant)', 'Zo(Characteristic Impedance)'] 

                                fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                                cutoff_frequency = 10  # 設定截止頻率 (Hz)
                                order = 2  # 濾波器階數
                                b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                                # 對每個輸出結果進行低通濾波
                                filtered_output = np.zeros_like(predicted_output)
                                for i in range(predicted_output.shape[1]):
                                    filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])



                                # 迭代每個數據列並繪製相應的圖形
                                for j in range(4):  # 假設 data_frame 有4列數據需要繪製
                                    fig, ax = plt.subplots()
                                    ax.plot(input_freq, filtered_output[:, j+1], label=f'Simulation data')
                                    ax.set_xlabel('Frequency (GHz)')
                                    ax.set_ylabel('Magnitude')
                                    ax.set_title(titles[j], fontsize=16)
                                    ax.legend()

                                    # 將圖形轉換為圖像數據
                                    fig.canvas.draw()
                                    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                    plt.close(fig)

                                    # 計算圖像的位置
                                    row = j // 2
                                    col = j % 2
                                    height, width, channel = image.shape
                                    bytesPerLine = 3 * width
                                    q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                    pixmap = QPixmap.fromImage(q_image)

                                    # 設置圖像到對應的 QLabel 中
                                    image_labels[row][col].setPixmap(pixmap.scaled(450, 300, Qt.KeepAspectRatio))


                else:
                    self.set_black_images(self.image_labels_tab1)
                    self.show_error_message("Please enter a valid number for frequency.")

            
            
            
        elif tab_name == "C1":
            if not self.input1_tab3.text() or not self.input2_tab3.text() or not self.input3_tab3.text() or not self.input4_tab3.text() or not self.input5_tab3.text() : 
                self.set_black_images(self.image_labels_tab1)
                self.show_error_message("Please enter all parameters .")
            else :
                input1 = float(self.input1_tab3.text())
                input2 = float(self.input2_tab3.text())
                via_d = float(self.input3_tab3.text())
                pad_d = float(self.input4_tab3.text())
                via_shield_d = float(self.input5_tab3.text())
                image_labels = self.image_labels_tab3

                if input1 >= 37 and input2 <= 41 and input2 > input1: 


                    if via_d < 0.1 or via_d > 0.25:
                        self.set_black_images(self.image_labels_tab1) #都要寫tab1 不然不會出現
                        self.show_error_message("Please enter a valid number for via .")
                    else :

                        via = via_d

                        if pad_d < 0.6 or pad_d > 1 :
                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for pad .")
                        else : 

                            pad = pad_d

                        if via_shield_d < 1.3 or via_shield_d > 1.7 :

                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for via shield .")
                        else :

                            shield = via_shield_d

                            input_freq = np.arange(input1, input2+0.01, 0.01)  # 從37到41，間隔0.01
                            size_input = input_freq.shape[0]
                            
                            input_via = np.full(size_input,via)  # 401個0.1
                            input_pad = np.full(size_input, pad/2)  # 401個0.3
                            input_shield = np.full(size_input, shield/2)  # 401個0.66
                
                
                            input_data = np.array([input_via, input_pad, input_shield, input_freq]).T

                            file_name = f"Trained_model/model_C1_mid.joblib"
                            loaded_multi_output_model = joblib.load(f"Trained_model/model_C1_mid.joblib")
                            predicted_output = loaded_multi_output_model.predict(input_data)
                            titles = ['S11(Return loss)','S21(Insertion loss)']
                            
                            fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                            cutoff_frequency = 15  # 設定截止頻率 (Hz)
                            order = 2  # 濾波器階數
                            b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                            # 對每個輸出結果進行低通濾波
                            filtered_output = np.zeros_like(predicted_output)
                            for i in range(predicted_output.shape[1]):
                                filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])

                            for j in range(2):  # 假設現在只需要繪製2個圖片
                                fig, ax = plt.subplots()
                                ax.plot(input_freq, filtered_output[:, j], label=f'Simulation data')
                                ax.set_xlabel('Frequency (GHz)')
                                ax.set_ylabel('Magnitude')
                                ax.set_title(titles[j], fontsize=16)
                                ax.legend()

                                # 將圖形轉換為圖像數據
                                fig.canvas.draw()
                                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                plt.close(fig)

                                # 計算圖像的位置
                                row = j  # 每個圖片都放在不同的row
                                height, width, channel = image.shape
                                bytesPerLine = 3 * width
                                q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                pixmap = QPixmap.fromImage(q_image)

                                # 設置圖像到對應的 QLabel 中
                                image_labels[row].setPixmap(pixmap.scaled(500, 300, Qt.KeepAspectRatio))


                else:
                    self.set_black_images(self.image_labels_tab1)
                    self.show_error_message("Please enter a valid number for frequency.")
        
        elif tab_name == "C2":
            if not self.input1_tab4.text() or not self.input2_tab4.text() or not self.input3_tab4.text() or not self.input4_tab4.text() or not self.input5_tab4.text() : 
                self.set_black_images(self.image_labels_tab1)
                self.show_error_message("Please enter all parameters .")
            else :
                input1 = float(self.input1_tab4.text())
                input2 = float(self.input2_tab4.text())
                via_d = float(self.input3_tab4.text())
                pad_d = float(self.input4_tab4.text())
                via_shield_d = float(self.input5_tab4.text())
                image_labels = self.image_labels_tab4

                if input1 >= 57 and input2 <= 63 and input2 > input1: 


                    if via_d < 0.1 or via_d > 0.3:
                        self.set_black_images(self.image_labels_tab1)
                        self.show_error_message("Please enter a valid number for via .")
                    else :

                        via = via_d

                        if pad_d < 0.2 or pad_d > 0.6 :
                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for pad .")
                        else : 

                            pad = pad_d

                        if via_shield_d < 1.3 or via_shield_d > 1.6 :

                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for via shield .")
                        else :

                            shield = via_shield_d
                            
                            input_freq = np.arange(input1, input2+0.01, 0.01)  # 從37到41，間隔0.01
                            size_input = input_freq.shape[0]
                            
                            input_via = np.full(size_input,via)  # 401個0.1
                            input_pad = np.full(size_input, pad/2)  # 401個0.3
                            input_shield = np.full(size_input, shield/2)  # 401個0.66
                
                            input_data = np.array([input_via, input_pad, input_shield, input_freq]).T

                    
                            file_name = f"Trained_model/model_C2_high.joblib"
                            loaded_multi_output_model = joblib.load(f"Trained_model/model_C2_high.joblib")
                            predicted_output = loaded_multi_output_model.predict(input_data)
                            titles = ['S11(Return loss)','S21(Insertion loss)']
                            
                                                        
                            fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                            cutoff_frequency = 10  # 設定截止頻率 (Hz)
                            order = 2  # 濾波器階數
                            b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                            # 對每個輸出結果進行低通濾波
                            filtered_output = np.zeros_like(predicted_output)
                            for i in range(predicted_output.shape[1]):
                                filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])

                            for j in range(2):  # 假設現在只需要繪製2個圖片
                                fig, ax = plt.subplots()
                                ax.plot(input_freq, filtered_output[:, j], label=f'Simulation data')
                                ax.set_xlabel('Frequency (GHz)')
                                ax.set_ylabel('Magnitude')
                                ax.set_title(titles[j], fontsize=16)
                                ax.legend()

                                # 將圖形轉換為圖像數據
                                fig.canvas.draw()
                                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                plt.close(fig)

                                # 計算圖像的位置
                                row = j  # 每個圖片都放在不同的row
                                height, width, channel = image.shape
                                bytesPerLine = 3 * width
                                q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                pixmap = QPixmap.fromImage(q_image)

                                # 設置圖像到對應的 QLabel 中
                                image_labels[row].setPixmap(pixmap.scaled(500, 300, Qt.KeepAspectRatio))


                else:
                    self.set_black_images(self.image_labels_tab1)
                    self.show_error_message("Please enter a valid number for frequency.")
        elif tab_name == "A1":
            if not self.input1_tab5.text() or not self.input2_tab5.text() or not self.input3_tab5.text() or not self.input4_tab5.text() or not self.input5_tab5.text() : 
                self.set_black_images(self.image_labels_tab1)
                self.show_error_message("Please enter all parameters .")
            else :
                input1 = float(self.input1_tab5.text())
                input2 = float(self.input2_tab5.text())
                via_d = float(self.input3_tab5.text())
                pad_d = float(self.input4_tab5.text())
                via_shield_d = float(self.input5_tab5.text())
                image_labels = self.image_labels_tab5

                if input1 >= 37 and input2 <= 41 and input2 > input1: 
                    freq_401 = np.arange(37,41.01,0.01)
                    freq_x_axis = np.arange(input1, input2 + 0.01, 0.01)
                    num_freq = freq_x_axis.shape[0] #需要多大的頻率範圍點數

                    if via_d < 0.1 or via_d > 0.3:

                        self.set_black_images(self.image_labels_tab1) #都要寫tab1 不然不會出現
                        self.show_error_message("Please enter a valid number for via .")
                    else :

                        via = via_d

                        if pad_d < 0.6 or pad_d > 0.8 :

                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for pad .")
                        else : 

                            pad = pad_d

                        if via_shield_d < 1.35 or via_shield_d > 1.5 :


                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for via shield .")
                        else :

                            shield = via_shield_d
                            
                            input_freq = np.arange(input1, input2+0.01, 0.01)

                            size_input = input_freq.shape[0]
                            
                            input_via = np.full(size_input,via)  
                            input_pad = np.full(size_input, pad)  
                            input_shield = np.full(size_input, shield)  
                
                            input_data = np.array([input_via, input_pad, input_shield, input_freq]).T

                            file_name = f"Trained_model/model_A1_mid.joblib"
                            loaded_multi_output_model = joblib.load(f"Trained_model/model_A1_mid.joblib")
                            predicted_output = loaded_multi_output_model.predict(input_data)
                            print(predicted_output.shape)
                            titles = ['S11(Return loss)','S21(Insertion loss)']

                            fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                            cutoff_frequency = 10  # 設定截止頻率 (Hz)
                            order = 2  # 濾波器階數
                            b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                            # 對每個輸出結果進行低通濾波
                            filtered_output = np.zeros_like(predicted_output)
                            for i in range(predicted_output.shape[1]):
                                filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])


                            for j in range(2):  # 假設現在只需要繪製2個圖片
                                fig, ax = plt.subplots()
                                ax.plot(input_freq, filtered_output[:, j], label=f'Simulation data')
                                ax.set_xlabel('Frequency (GHz)')
                                ax.set_ylabel('Magnitude')
                                ax.set_title(titles[j], fontsize=16)
                                ax.legend()

                                # 將圖形轉換為圖像數據
                                fig.canvas.draw()
                                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                plt.close(fig)

                                # 計算圖像的位置
                                row = j  # 每個圖片都放在不同的row
                                height, width, channel = image.shape
                                bytesPerLine = 3 * width
                                q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                pixmap = QPixmap.fromImage(q_image)

                                # 設置圖像到對應的 QLabel 中
                                image_labels[row].setPixmap(pixmap.scaled(500, 300, Qt.KeepAspectRatio))


                else:
                    self.set_black_images(self.image_labels_tab1)
                    self.show_error_message("Please enter a valid number for frequency.")
        
        elif tab_name == "A2":
            if not self.input1_tab6.text() or not self.input2_tab6.text() or not self.input3_tab6.text() or not self.input4_tab6.text() or not self.input5_tab6.text() : 
                self.set_black_images(self.image_labels_tab1)
                self.show_error_message("Please enter all parameters .")
            else : 
                input1 = float(self.input1_tab6.text())
                input2 = float(self.input2_tab6.text())
                via_d = float(self.input3_tab6.text())
                pad_d = float(self.input4_tab6.text())
                via_shield_d = float(self.input5_tab6.text())
                image_labels = self.image_labels_tab6

                if input1 >= 57 and input2 <= 63 and input2 > input1: 
                    freq_401 = np.arange(57,63.01,0.01)
                    freq_x_axis = np.arange(input1, input2 + 0.01, 0.01)
                    num_freq = freq_x_axis.shape[0] #需要多大的頻率範圍點數

                    if via_d < 0.1 or via_d > 0.25:
                        self.set_black_images(self.image_labels_tab1)
                        self.show_error_message("Please enter a valid number for via .")
                    else :

                        via = via_d

                        if pad_d < 0.6 or pad_d > 0.8 :

                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for pad .")
                        else : 

                            pad = pad_d

                        if via_shield_d < 1.15 or via_shield_d > 1.35 :

                            self.set_black_images(self.image_labels_tab1)
                            self.show_error_message("Please enter a valid number for via shield .")
                        else :

                            shield = via_shield_d


                            input_freq = np.arange(input1, input2+0.01, 0.01)

                            size_input = input_freq.shape[0]
                            
                            input_via = np.full(size_input,via)  
                            input_pad = np.full(size_input, pad)  
                            input_shield = np.full(size_input, shield)  
                
                            input_data = np.array([input_via, input_pad, input_shield, input_freq]).T

                            file_name = f"Trained_model/model_A2_high.joblib"
                            loaded_multi_output_model = joblib.load(f"Trained_model/model_A2_high.joblib")
                            predicted_output = loaded_multi_output_model.predict(input_data)
                            print(predicted_output.shape)
                            titles = ['S11(Return loss)','S21(Insertion loss)']

                            fs = 1 / 0.01  # 計算採樣頻率 (Hz)
                            cutoff_frequency = 10  # 設定截止頻率 (Hz)
                            order = 2  # 濾波器階數
                            b, a = butter(order, cutoff_frequency / (fs / 2), btype='low')

                            # 對每個輸出結果進行低通濾波
                            filtered_output = np.zeros_like(predicted_output)
                            for i in range(predicted_output.shape[1]):
                                filtered_output[:, i] = filtfilt(b, a, predicted_output[:, i])


                            for j in range(2):  # 假設現在只需要繪製2個圖片
                                fig, ax = plt.subplots()
                                ax.plot(input_freq, filtered_output[:, j], label=f'Simulation data')

                                ax.set_xlabel('Frequency (GHz)')
                                ax.set_ylabel('Magnitude')
                                ax.set_title(titles[j], fontsize=16)
                                ax.legend()

                                # 將圖形轉換為圖像數據
                                fig.canvas.draw()
                                image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                                image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                                plt.close(fig)

                                # 計算圖像的位置
                                row = j  # 每個圖片都放在不同的row
                                height, width, channel = image.shape
                                bytesPerLine = 3 * width
                                q_image = QImage(image.data, width, height, bytesPerLine, QImage.Format_RGB888)
                                pixmap = QPixmap.fromImage(q_image)

                                # 設置圖像到對應的 QLabel 中
                                image_labels[row].setPixmap(pixmap.scaled(500, 300, Qt.KeepAspectRatio))


                else:
                    self.set_black_images(self.image_labels_tab1)
                    self.show_error_message("Please enter a valid number for frequency.")

            
    def set_black_images(self, image_labels): #如果有錯誤 ->黑頻
        black_image = QPixmap(300, 250)
        black_image.fill(Qt.black)
        for row in image_labels:
            for label in row:
                label.setPixmap(black_image)   
    def show_error_message(self, message):
        error_dialog = QErrorMessage(self)
        error_dialog.showMessage(message)

            
# Parameters to control the layout and size
image_size = (450, 300)  # Image size (width, height)
button_size = (120, 40)  # Button size (width, height)

def create_main_window():
    app = QApplication(sys.argv)
    window = MainWindow()

    # Adjusting the image sizes
    for tab_images in [window.image_labels_tab1, window.image_labels_tab2, window.image_labels_tab3, window.image_labels_tab4, window.image_labels_tab5, window.image_labels_tab6]:
        if isinstance(tab_images[0], list):
            for row in tab_images:
                for label in row:
                    label.setFixedSize(*image_size)
        else:
            for label in tab_images:
                label.setFixedSize(*image_size)
    
    # Adjusting the button size 
    for button in [window.calc_button_tab1, window.calc_button_tab2, window.calc_button_tab3,window.calc_button_tab4, window.calc_button_tab5, window.calc_button_tab6]:
        button.setFixedSize(*button_size)

    window.show()
    sys.exit(app.exec_())

create_main_window()
